In this study I read some URLs, parse some contents of them and apply Kmeans clustering to cluster them
Notes:
- I did not have enough time to do this project in JAVA
- Not the whole "useful" content of the URls are being read
- I obviously used some libraries but I hope I know the concepts behind the methods :)
- Java is my strong language

In [1]:
import urllib
from BeautifulSoup import BeautifulSoup
import numpy as np
import scipy.misc
import mglearn 

In [2]:
links = ["https://www.webmd.com/digestive-disorders/diverticular-disease#1",
         "https://www.washingtonpost.com/politics/trump-claims-his-asia-trip-forged-new-unity-against-north-korea/2017/11/15/7961639e-ca42-11e7-b0cf-7689a9f2d84e_story.html?utm_term=.14aff79f6a7d",
         "https://www.theatlantic.com/politics/archive/2017/11/the-secret-correspondence-between-donald-trump-jr-and-wikileaks/545738/",
         "https://mobile.nytimes.com/2017/11/15/us/politics/senate-house-tax-cut.html",
         "https://www.fascrs.org/patients/disease-condition/diverticular-disease",
        ]



In [3]:
data = []
for link in links:
    f = urllib.urlopen(link)
    f_read = f.read()
    parsed_html = BeautifulSoup(f_read)
    body = parsed_html.body.find('div', attrs={'class':['article-body',
                                                   'article-page active-page',
                                                   'story-body-text story-content',
                                                   'story-body story-body-2',
                                                   'field-item even'
                                                  ]}).text
    data.append(body)





In [4]:
len(data)

5

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

Here we use the tfidf (term frq- inverse doc frequency) vectorizer
We can pass the right parameters to the constructor to:
- remove the stop words
- to define the range of the frequency we wanna filter from this analysis
- to tokenize and stem
- ngram range 
- normalization ? ( I use norm 2 for Kmean algorithm)

In [6]:

vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.2, stop_words='english', norm = 'l2')

tfdif_matrix = vectorizer.fit_transform(data)

The whole point of vectorization was make it easy to find some distance.
To be more particular:
The distance is : 1 - COSINE(tfdfif matrix)

In [7]:
from sklearn.cluster import KMeans

In [8]:
kmeans = KMeans(n_clusters=3)
topics = kmeans.fit(tfdif_matrix)

In [9]:
kmeans.labels_

array([0, 1, 1, 2, 0], dtype=int32)

In [10]:
features = np.array(vectorizer.get_feature_names())
sorting = np.argsort(kmeans.cluster_centers_, axis = 1)[:,::-1]

Let's print out the topics here:


In [11]:
mglearn.tools.print_topics(topics=range(3), feature_names = features, sorting = sorting,
                           topics_per_chunk = 3, n_words = 10)

topic 0       topic 1       topic 2       
--------      --------      --------      
colon         trump         tax           
diverticulitiswikileaks     republicans   
diverticulosisjr            senate        
surgery       north         republican    
symptoms      korea         health        
diverticula   said          senators      
nbsp          wrote         care          
diverticular  did           house         
infection     korean        mr            
bleeding      america       repeal        




I think the results sort of make sense. The first cluster contains the specific health issue.
The second and third clusters are politics related and tax reform and healthcare.
